# Unpacking Original VotePageInfo.json File

## This python notebook packages a series of steps into one function, which results in a new pickled Dataframe object file being generated with the unpacked json data

## The final helper function has 3 internal steps:

### 1) Import the JSON file into a Pandas notebook
### 2) Unpack the nested JSON objects into indivdual dataframe columns, and delete the old, nested columns
### 3) Save the new pickled file (which is easier to import and work with)

In [1]:
import pandas as pd
import json
import pickle

## 1) Import the Routes Json File

In [56]:
def import_json_file(filepath):
    # Import file into pandas dataframe
    with open(filepath) as file:
        reader = file.read()
        votesDict = dict(json.loads(reader))
        votes = pd.DataFrame.from_dict(votesDict, orient='index')
    return votes

### Functions to extract Data

In [57]:
# Extract the user star votes and return a list of lists with format"
# [ ['userUrl_1_climb1', 'User_1_Vote_climb1'], ['userUrl_2_climb1', 'User_2_Vote_climb1'], ...]
def get_user_star_votes(votes_dataframe):
    votes_dataframe['userStarVotes'] = [votes_dataframe['starRatings'][item]['ratings'] for item in range(len(votes_dataframe['starRatings']))]
    userStarVotes = []
    it = iter(votes_dataframe)
    for i in it:
        print(i)
    for vote in votes_dataframe.userStarVotes:
        climbVotes = []
        try:
            for item in vote:
                climbVotes.append([item, vote[item]['rating']])
            userStarVotes.append(climbVotes)
        except:
            userStarVotes.append(None)
    votes_dataframe.drop(['userStarVotes'], axis=1)

    return userStarVotes

# Extract the difficulty ratings and return a list of lists
def get_difficulty_ratings(votes_dataframe):
    votes_dataframe['diffRatings'] = [votes_dataframe['difficultyRatings'][item]['ratings'] for item in range(len(votes_dataframe['difficultyRatings']))]
    difficultyRatings = []
    for vote in votes_dataframe.diffRatings:
        climbVotes = []
        try:
            for item in vote:
                climbVotes.append([item, vote[item]['rating']])
            difficultyRatings.append(climbVotes)
        except:
            difficultyRatings.append(climbVotes)
    
    return difficultyRatings

# Extract a route's ticks as a list of dictionaries. Each list member is one dictionary
def get_ticks(votes_dataframe):
    votes_dataframe['ticks'] = [votes_dataframe['ticks'][item] for item in range(len(votes_dataframe['ticks']))]
    ticks = []
    for route in votes_dataframe['ticks']:
        ticks.append(route)
    return ticks
            
# Extract a route's ticks as a list of dictionaries. Each list member is one dictionary
def get_toDo_list(votes_dataframe):
    votes_dataframe['toDoList'] = [votes_dataframe['toDoList'][item] for item in range(len(votes_dataframe['toDoList']))]
    toDoList = []
    for route in votes_dataframe['toDoList']:
        toDoList.append(route['users'])
    return toDoList

# Get the number of items in each field, and save as a new field
def add_count_column(votes, newColumnName, fieldToCount):
    countColumn = []
    for item in range(len(votes)):
        try:
            countColumn.append(len(votes[fieldToCount][item]))
        except:
            countColumn.append(0)
    votes[newColumnName] = countColumn
        
    return votes

# Get the list of people who voted. Currently only works for star votes
def add_list_of_voters(votes, newColumnName, fieldToScan):
    users = []
    for item in range(len(votes)):
        try:
            specificRoute = []
            for user in range(len(votes[fieldToScan][item])):
                specificRoute.append(votes[fieldToScan][item][user][0])
            users.append(specificRoute)
        except:
            users.append(0)
    votes[newColumnName] = users
    return votes

## 2) Unpack nested JSON objects

In [58]:
# Get the data
def unpack_data(votes):
    votes['starRatings'] = get_user_star_votes(votes)
    votes['diffRatings'] = get_difficulty_ratings(votes)
    votes['ticks'] = get_ticks(votes)
    votes['toDoList'] = get_toDo_list(votes)
    votes.drop(columns=['userStarVotes', 'diffRatings'], axis=1, inplace=True)
    votes = add_count_column(votes, 'starRatingCount', 'starRatings')
    votes = add_list_of_voters(votes, 'starRatingUsers', 'starRatings')
    return votes

## 3) Save new file with unpacked JSON objects as individual columns

In [59]:
# Save the newly unpacked votes file
def save_vote_dataframe(votes_dataframe, destPath):
    votes_dataframe.to_pickle(destPath)

## 4) Package into a helper function to do 1, 2 and 3 in one step

In [60]:
def unpack_votes(filepath, destPath):
    votes = import_json_file(filepath)
    votes = unpack_data(votes)
    save_vote_dataframe(votes, destPath)

filepath = './Data/votePageData.json'
destPath = './Data/pickledVotes_unpacked'

name
starRatings
difficultyRatings
toDoList
ticks
userStarVotes


In [61]:
votes = import_json_file(filepath)
votes = unpack_data(votes)

name
starRatings
difficultyRatings
toDoList
ticks
userStarVotes


# Development Code is Below

In [8]:
votes.head(5)

,name,starRatings,difficultyRatings,toDoList,ticks
https://www.mountainproject.com/route/105714687/bridalveil-falls-right-side,Side,[[https://www.mountainproject.com/user/13248/j...,[[https://www.mountainproject.com/user/1062091...,{'https://www.mountainproject.com/user/13914/t...,"{'quantity': '13', 'tickList': {'https://www.m..."
https://www.mountainproject.com/route/105714691/honey-bear-main-flow,Flow,[[https://www.mountainproject.com/user/1063883...,[[https://www.mountainproject.com/user/1063883...,{'https://www.mountainproject.com/user/1062091...,"{'quantity': '6', 'tickList': {'https://www.mo..."
https://www.mountainproject.com/route/105714695/terminator-pillar,Pillar,[[https://www.mountainproject.com/user/1076300...,[[https://www.mountainproject.com/user/1063546...,{'https://www.mountainproject.com/user/13914/t...,"{'quantity': '7', 'tickList': {'https://www.mo..."
https://www.mountainproject.com/route/105714699/11th-hour-center,Center,[[https://www.mountainproject.com/user/1066034...,[[https://www.mountainproject.com/user/1063546...,{'https://www.mountainproject.com/user/13914/t...,"{'quantity': '10', 'tickList': {'https://www.m..."
https://www.mountainproject.com/route/105714703/gorillas-in-the-schist,Schist,[[https://www.mountainproject.com/user/1058628...,[],None,"{'quantity': None, 'tickList': None}"


In [52]:
# Add column to votes DF for # of ratings/diffRatings/To-Do Lists/Ticks
# try:
#     votes['numStarRatings'] = [len(votes['starRatings'][item]) for item in range(len(votes))]
# except:
#     pass
# try:
#     votes['numDiffRatings'] = [len(votes['difficultyRatings'][item]) for item in range(len(votes))]
# except:
#     pass
# try:
#     votes['numToDoLists'] = [len(votes['toDoList'][item]) for item in range(len(votes))]
# except:
#     pass
# try:
#     votes['numTicks'] = [len(votes['ticks'][item]) for item in range(len(votes))]
# except:
#     pass
    
# Add column to votes DF for all users who voted/toDoList/etc'



In [19]:
votes = add_count_column(votes, 'starRatingCount', 'starRatings')
votes = add_count_column(votes, 'diffRatingCount', 'difficultyRating')
votes = add_count_column(votes, 'toDoListCount', 'toDoList')
votes = add_count_column(votes, 'ticksCount', 'ticks')

188534
188534
188534
188534
188534
188534
188534
188534


In [53]:
votes = add_list_of_voters(votes, 'starRatingUsers', 'starRatings')

In [54]:
votes.iloc[0]['starRatingUsers']

['https://www.mountainproject.com/user/13248/john-gunnels',
 'https://www.mountainproject.com/user/10246/andrew-gram',
 'https://www.mountainproject.com/user/106209198/markus-jobman',
 'https://www.mountainproject.com/user/107630068/clay-cundy',
 'https://www.mountainproject.com/user/107048400/mike-vetter']

In [55]:
votes.loc['https://www.mountainproject.com/route/105714703/gorillas-in-the-schist']

name                                                            Schist
starRatings          [[https://www.mountainproject.com/user/1058628...
difficultyRatings                     {'quantity': '2', 'ratings': {}}
toDoList                                                          None
ticks                             {'quantity': None, 'tickList': None}
starRatingCount                                                      2
diffRatingCount                                                      0
toDoListCount                                                        0
ticksCount                                                           2
starRatingUsers      [https://www.mountainproject.com/user/10586283...
Name: https://www.mountainproject.com/route/105714703/gorillas-in-the-schist, dtype: object

In [51]:
votes.head(5)

,name,starRatings,difficultyRatings,toDoList,ticks,starRatingCount,diffRatingCount,toDoListCount,ticksCount,starRatingUsers
https://www.mountainproject.com/route/105714687/bridalveil-falls-right-side,Side,[[https://www.mountainproject.com/user/13248/j...,"{'quantity': '3', 'ratings': {'https://www.mou...",{'https://www.mountainproject.com/user/13914/t...,"{'quantity': '13', 'tickList': {'https://www.m...",5,0,3,2,[[https://www.mountainproject.com/user/13248/j...
https://www.mountainproject.com/route/105714691/honey-bear-main-flow,Flow,[[https://www.mountainproject.com/user/1063883...,"{'quantity': '4', 'ratings': {'https://www.mou...",{'https://www.mountainproject.com/user/1062091...,"{'quantity': '6', 'tickList': {'https://www.mo...",6,0,4,2,[[https://www.mountainproject.com/user/13248/j...
https://www.mountainproject.com/route/105714695/terminator-pillar,Pillar,[[https://www.mountainproject.com/user/1076300...,"{'quantity': '4', 'ratings': {'https://www.mou...",{'https://www.mountainproject.com/user/13914/t...,"{'quantity': '7', 'tickList': {'https://www.mo...",6,0,6,2,[[https://www.mountainproject.com/user/13248/j...
https://www.mountainproject.com/route/105714699/11th-hour-center,Center,[[https://www.mountainproject.com/user/1066034...,"{'quantity': '6', 'ratings': {'https://www.mou...",{'https://www.mountainproject.com/user/13914/t...,"{'quantity': '10', 'tickList': {'https://www.m...",9,0,3,2,[[https://www.mountainproject.com/user/13248/j...
https://www.mountainproject.com/route/105714703/gorillas-in-the-schist,Schist,[[https://www.mountainproject.com/user/1058628...,"{'quantity': '2', 'ratings': {}}",None,"{'quantity': None, 'tickList': None}",2,0,0,2,[[https://www.mountainproject.com/user/13248/j...
